In [1]:
import numpy as np

from data_utils import compute_pairs_euclidean_distances, load_numpy_data, save_numpy_array

from psf_constants import FC_PROCESSED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                          FC_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                          FC_PREDICTED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                          FC_PREDICTED_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, \
                          FC_PROCESSED_TRAIN_OUTPUT_FLUXES_PREFIX, \
                          TRAIN_EUCLIDEAN_DISTANCES_PREFIX, \
                          TRAIN_EUCLIDEAN_DISTANCES_PAIRS_PREFIX, \
                          PSF_TRAIN_FILE_SUFFIXES, \
                          NUMPY_SUFFIX

In [2]:
def create_path(prefix, suffix):
    data_path = f"{prefix}{suffix}{NUMPY_SUFFIX}"
    return data_path

def create_random_pair_indexes(array_n_points, pairs_per_subset=10000):
    selected_pairs = np.random.randint(0, array_n_points, size=(pairs_per_subset, 2))
    selected_pairs = selected_pairs[selected_pairs[:, 0] != selected_pairs[:, 1]]
    
    while selected_pairs.shape[0] < pairs_per_subset:
        more_pairs = np.random.randint(0, 100, size=(pairs_per_subset, 2))
        more_pairs = more_pairs[more_pairs[:, 0] != more_pairs[:, 1]]
        selected_pairs = np.concatenate((selected_pairs, more_pairs))
        
    return selected_pairs[:pairs_per_subset]

# 1. Brute Force

In [3]:
for suffix in PSF_TRAIN_FILE_SUFFIXES:

    # Define data paths
    fluxes_path = create_path(FC_PROCESSED_TRAIN_OUTPUT_FLUXES_PREFIX, suffix)
    og_complex_fields_path = create_path(FC_PROCESSED_TRAIN_COMPLEX_FIELDS_PREFIX, suffix)
    og_cropped_complex_fields_path = create_path(FC_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, suffix)
    predicted_complex_fields_path = create_path(FC_PREDICTED_TRAIN_COMPLEX_FIELDS_PREFIX, suffix)
    predicted_cropped_complex_fields_path = create_path(FC_PREDICTED_CROPPED_TRAIN_COMPLEX_FIELDS_PREFIX, suffix)

    # Load data
    psf_fluxes = load_numpy_data(fluxes_path)
    og_complex_fields = load_numpy_data(og_complex_fields_path)
    og_cropped_complex_fields = load_numpy_data(og_cropped_complex_fields_path)
    predicted_complex_fields = load_numpy_data(predicted_complex_fields_path)
    predicted_cropped_complex_fields = load_numpy_data(predicted_cropped_complex_fields_path)

    # Select pairs to compute euclidean distances
    n_points = psf_fluxes.shape[0]
    selected_pairs = create_random_pair_indexes(n_points, pairs_per_subset=10000)

    # Compute euclidean distances
    psf_fluxes_euclidean_distances = compute_pairs_euclidean_distances(psf_fluxes,
                                                                       selected_pairs)
    
    og_euclidean_distances = compute_pairs_euclidean_distances(og_complex_fields,
                                                               selected_pairs,
                                                               is_complex_field=True)
    
    og_cropped_euclidean_distances = compute_pairs_euclidean_distances(og_cropped_complex_fields,
                                                                       selected_pairs,
                                                                       is_complex_field=True)
    
    predicted_euclidean_distances = compute_pairs_euclidean_distances(predicted_complex_fields, 
                                                                      selected_pairs,
                                                                      is_complex_field=True)
    
    predicted_cropped_euclidean_distances = compute_pairs_euclidean_distances(predicted_cropped_complex_fields,
                                                                              selected_pairs,
                                                                              is_complex_field=True)

    # Merge in columns
    euclidean_distances = np.concatenate((psf_fluxes_euclidean_distances.reshape(-1, 1),
                                          og_euclidean_distances.reshape(-1, 1),
                                          og_cropped_euclidean_distances.reshape(-1, 1),
                                          predicted_euclidean_distances.reshape(-1, 1),
                                          predicted_cropped_euclidean_distances.reshape(-1, 1)), 
                                          axis=1)

    # Save arrays
    pairs_path = create_path(TRAIN_EUCLIDEAN_DISTANCES_PAIRS_PREFIX, suffix)
    euclidean_distances_path = create_path(TRAIN_EUCLIDEAN_DISTANCES_PREFIX, suffix)
    save_numpy_array(selected_pairs, pairs_path, single_precision=False)
    save_numpy_array(euclidean_distances, euclidean_distances_path)

In [7]:
og_cropped_euclidean_distances.min()

11.103067398071289